In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model (ensure the model path is correct)
model = load_model("mnist_trained_model.keras")  # Make sure you have the correct model path

# Initialize webcam
cap = cv2.VideoCapture(0)  # 0 is the default webcam (change to 1 if using a different webcam)

# Set the resolution of the webcam (optional)
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

# Define the function to preprocess the captured image
def preprocess_image(image):
    """
    Preprocesses the captured image to be ready for prediction:
    - Resize to 28x28 (MNIST image size)
    - Normalize pixel values
    """
    # Resize the image to 28x28 (standard input size for MNIST)
    resized_image = cv2.resize(image, (28, 28), interpolation=cv2.INTER_AREA)
    
    # Normalize the image (scaling pixel values to the range [0, 1])
    resized_image = resized_image.astype("float32") / 255.0
    
    # Reshape the image to match the model input shape: (1, 28, 28, 1)
    resized_image = np.expand_dims(resized_image, axis=0)  # Add batch dimension
    resized_image = np.expand_dims(resized_image, axis=-1)  # Add channel dimension (for grayscale)
    
    return resized_image

# Main loop to capture images from webcam
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to capture image")
        break
    
    # Convert the captured frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to remove noise and improve contour detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply adaptive thresholding instead of a fixed threshold value
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours of the digits
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        # Get the bounding box of each contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Filter out small contours that are unlikely to be digits
        if w > 20 and h > 20:
            # Extract the ROI (Region of Interest) containing the digit
            roi = thresh[y:y+h, x:x+w]
            
            # Resize the ROI to 28x28 for prediction
            processed_image = preprocess_image(roi)
            
            # Predict the digit using the model
            prediction = model.predict(processed_image)
            predicted_digit = np.argmax(prediction)  # Get the digit with the highest probability
            confidence = np.max(prediction)  # Highest probability

            # Draw the bounding box and predicted digit on the frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"Pred: {predicted_digit}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Conf: {confidence:.2f}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Show the live webcam feed with predictions
    cv2.imshow("Webcam Feed", frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━